In [1]:
from get_server_data import get_db_table_as_df, db_filename
import sqlite3
import pandas as pd
import numpy as np

In [2]:
flood_locations = get_db_table_as_df('flood_locations')
flood_events = get_db_table_as_df('flood_events', date_col=['event_date', 'dates'])
for_model = get_db_table_as_df('for_model', date_col=['event_date'])
daily_obs = get_db_table_as_df('dntwn_nor_daily_observations', date_col=['Datetime'])

In [3]:
flood_locations.head()

,index,elev,disttowat,location,eventType,count,xcoord,ycoord,long,lat,flood_pt,num_imperv,is_downtown,distance_to_OW,distance_to_OW_WW,distance_to_OW_EW
0,0,0.663456,353.260010,1500 BLOCK OF LEA VIEW AVENUE,Flooded street,1,1.212703e+07,3.519555e+06,-76.295029,36.968341,1,74738,0,383.939112,383.939112,383.939112
1,1,1.091590,615.931030,,,0,1.213003e+07,3.519171e+06,-76.284784,36.967096,0,101539,0,NaN,NaN,NaN
2,2,1.307290,643.104981,,,0,1.212960e+07,3.519094e+06,-76.286271,36.966911,0,109302,0,NaN,NaN,NaN
3,3,0.641102,354.657989,1503 LEA VIEW AVENUE,Flooded street,1,1.212737e+07,3.519508e+06,-76.293869,36.968192,1,78962,0,481.531796,481.531796,481.531796
4,4,0.877023,505.970001,9700 BLOCK OF 15TH VIEW STREET,Flooded street,1,1.212744e+07,3.519342e+06,-76.293648,36.967730,1,90165,0,610.225025,610.225025,610.225025


In [4]:
daily_obs.set_index('Datetime', inplace=True)
daily_obs.head()

,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_15_min_max,rain_15_min_max_time,rain_prev_3_days,gw_daily_avg,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg
Datetime,,,,,,,,,,,,,
2010-01-01,0.055,0.04,2010-01-01 22:45:00,0.02,2010-01-01 22:30:00,0.145,3.104508,0.387667,1.654,1.581,353.0,6.75,NaN
2010-01-02,0.000,0.00,None,0.00,None,0.130,3.414894,0.352125,NaN,NaN,321.0,16.90,NaN
2010-01-03,0.000,0.00,None,0.00,None,0.055,3.120831,-0.853333,NaN,NaN,295.0,13.90,NaN
2010-01-04,0.050,0.10,2010-01-04 13:30:00,0.10,2010-01-04 13:30:00,0.050,2.948871,-0.789292,0.279,0.279,306.0,8.50,NaN
2010-01-05,0.000,0.00,None,0.00,None,0.050,2.833045,-0.235708,NaN,NaN,292.0,7.25,NaN


In [5]:
daily_obs.shape

(2496, 13)

In [6]:
flood_events.head()

,location,eventType,event_name,event_date,dates,times
0,300 BLOCK OF SHIRLEY AVENUE,Flooded street,Nicole,2010-09-30,2010-09-30,07:32:47.000
1,LLEWELLYN AVENUE & SHIRLEY AVENUE,Flooded street,Nicole,2010-09-30,2010-09-30,07:33:19.000
2,LLEWELLYN AVENUE & W PRINCESS ANNE ROAD,Flooded street,Nicole,2010-09-30,2010-09-30,07:33:48.000
3,900 BLOCK OF TIDEWATER DRIVE,Flooded street,Nicole,2010-09-30,2010-09-30,08:29:00.000
4,CAMPOSTELLA ROAD & E BRAMBLETON AVENUE,Flooded street,Nicole,2010-09-30,2010-09-30,08:29:38.000


In [7]:
for_model.set_index('event_date', inplace=True)
for_model.head()

,rain_daily_sum,rain_hourly_max,rain_hourly_max_time,rain_15_min_max,rain_15_min_max_time,rain_prev_3_days,gw_daily_avg,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg,num_flooded,flooded,event_name
event_date,,,,,,,,,,,,,,,,
2010-09-30,11.415000,1.59,2010-09-30 04:15:00,0.67,2010-09-30 15:45:00,11.695000,3.051157,1.111472,0.961,0.577,291.666667,13.200000,NaN,48.0,1,Nicole
2011-08-27,7.895000,1.22,2011-08-27 11:30:00,0.33,2011-08-27 11:00:00,8.245000,1.806668,1.493500,2.602,2.516,155.500000,31.000000,NaN,32.0,1,Irene
2012-10-28,3.795000,0.58,2012-10-28 22:15:00,0.21,2012-10-28 21:45:00,2.495000,2.647606,2.903833,4.245,4.245,148.858198,23.504701,22.147917,45.0,1,Sandy
2013-10-09,4.320000,0.50,2013-10-09 15:00:00,0.22,2013-10-10 01:15:00,3.825000,1.661222,1.949806,3.839,2.959,39.311475,19.630806,18.424167,6.0,1,Heavy Rain
2014-05-16,3.853333,2.23,2014-05-16 08:00:00,0.76,2014-05-16 07:30:00,3.883333,2.208672,0.328958,0.643,0.643,214.833146,6.531076,8.279167,21.0,1,Heavy Rain


In [8]:
flood_locations.shape

(1547, 16)

In [9]:
lst = []
for index, row in flood_locations.iterrows():
    l = row['location'].strip()
    loc_data = daily_obs.copy()
    loc_data['flooded'] = 0
    if l != '':
        event_dates = flood_events[flood_events['location'] == l]['event_date']
        all_fld_dates = flood_events[flood_events['event_date'].isin(event_dates)]['dates']
        all_fld_dates = np.append(all_fld_dates, event_dates)
        all_fld_dates = pd.to_datetime(np.unique(all_fld_dates))

        fld_data = for_model[for_model.index.isin(event_dates)]
    
        loc_data = loc_data[~loc_data.index.isin(all_fld_dates)]
        loc_data = pd.concat([loc_data, fld_data])
    
        # add geog data
    for k in row.keys():
        loc_data[k] = row[k]
    lst.append(loc_data)
#         print l
#         print loc_data.head()
        

In [10]:
all_locations = pd.concat(lst)
all_locations.reset_index(inplace=True)

In [11]:
all_locations.shape

(3858155, 33)

In [12]:
new_cols = all_locations.columns.tolist()
new_cols[0] = 'Date'
all_locations.columns = new_cols
print all_locations.shape
all_locations.tail()

(3858155, 33)


,Date,count,distance_to_OW,distance_to_OW_EW,distance_to_OW_WW,disttowat,elev,eventType,event_name,flood_pt,...,rain_hourly_max_time,rain_prev_3_days,tide_daily_avg,tide_r15mx,tide_rhrmx,wind_dir_daily_avg,wind_vel_daily_avg,wind_vel_hourly_max_avg,xcoord,ycoord
3858150,2016-10-27,0,NaN,NaN,NaN,2674.23999,3.13519,,NaN,0,...,2016-10-27 22:15:00,1.750000e-02,0.083750,-0.026,-0.026,192.090694,7.817576,9.105833,1.212833e+07,3.486025e+06
3858151,2016-10-28,0,NaN,NaN,NaN,2674.23999,3.13519,,NaN,0,...,2016-10-28 04:30:00,9.250000e-02,0.194583,-0.394,-0.394,245.695972,8.178042,10.481250,1.212833e+07,3.486025e+06
3858152,2016-10-29,0,NaN,NaN,NaN,2674.23999,3.13519,,NaN,0,...,None,6.333333e-02,0.049375,NaN,NaN,212.104764,5.876687,6.940417,1.212833e+07,3.486025e+06
3858153,2016-10-30,0,NaN,NaN,NaN,2674.23999,3.13519,,NaN,0,...,None,5.000000e-02,-0.051250,NaN,NaN,241.199313,7.708653,9.345417,1.212833e+07,3.486025e+06
3858154,2016-10-31,0,NaN,NaN,NaN,2674.23999,3.13519,,NaN,0,...,None,-2.359802e-15,0.511737,NaN,NaN,40.838160,10.701056,13.190000,1.212833e+07,3.486025e+06


In [13]:
con = sqlite3.connect(db_filename)
all_locations.to_sql(con=con, name='for_model_geog', if_exists='replace')